# Assignment 2
## 1. Properties of CNNs
### Question 1.2 (a)

In this first part, we will train a CNN model over the dataset CIFAR-10. This dataset contains 10 classes: plane, car, bird, cat, deer, dog, frog, horse, ship and truck. 

First, let's load the data:

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [ ]:
# Import data from CIFAR-10
# Define device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),])

# Load CIFAR-10 dataset
train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

Create a function to display the first 25 images of a dataset with their corresponding class name and use it over `train_data`. 

Hint: `class_names = ["plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]`, where the labels are the corresponding index. 
   

In [ ]:
def display_first_few_images(data):
    #######################
    # PUT YOUR CODE HERE  #
    #######################
    category_labels = ["plane", "car", "bird", "cat", "deer",
                       "dog", "frog", "horse", "ship", "truck"]

    plt.figure(figsize=(8, 8))

    # go through the first 25 samples
    for idx in range(25):
        picture, target = data[idx]

        # convert from [channels, height, width] to an image format matplotlib understands
        image_to_show = picture.permute(1, 2, 0).numpy()

        plt.subplot(5, 5, idx + 1)
        plt.imshow(image_to_show)
        plt.title(category_labels[target])
        plt.axis("off")

    plt.tight_layout()
    plt.show() 
    #######################
    # END OF YOUR CODE    #
    #######################

In [ ]:
display_first_few_images(train_data)

Now, we will load and train a small CNN model! In the following cells you can check the architecture of the model and the designed function for its training.

Note: You shouldn't need Snellius to run it. Either Google Collab or your local computer should be enough.

In [ ]:
# Define the model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 4 * 4, 64)
        self.fc2 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Function for training the model
def train(model, train_loader, epochs = 10):
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
# Initialize the model 
model = CNN().to(device)

# Train the model
train(model, train_loader)

Now, we want to check the performance of the trained model for the test dataset when rotating the images 0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330 and 360 degrees.

For this, you need to first create the function `get_acc_per_angle` that computes the accuracies_per_angle for a given model. You should create a `rotated_test_data` and a `rotated_test_loader`, from which taking the images and labels to give as input to the `inference` function that is provided to you in the following cell. 

Then, create a `plot` function that plots the accuracy of the model per angle of rotation of the images.   

Hint: Check how we used transformations in section a.

In [ ]:
def inference(model, images, labels):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predictions = torch.max(outputs, 1)
    return predictions

In [ ]:
def get_acc_per_angle(model):
    angles = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360]
    angle_accuracies = []

    with torch.no_grad():
        for angle in angles:
            #######################
            # PUT YOUR CODE HERE  #
            #######################
            # for each angle I build a version of the CIFAR-10 test set

            # where every sample is rotated exactly by this angle
            rotation_transform = transforms.Compose([  # type: ignore
                transforms.RandomRotation((angle, angle)),  # force fixed rotation
                transforms.ToTensor(),                     # convert to tensor
            ])

            # load the normal test set but with my custom rotation applied
            rotated_test_data = datasets.CIFAR10(
                root='./data',
                train=False,
                download=False,
                transform=rotation_transform
            )

            # batching the rotated test set, no shuffle since it's evaluation
            rotated_test_loader = torch.utils.data.DataLoader(
                rotated_test_data,
                batch_size=64,
                shuffle=False
            )

            correct = 0
            total = 0

            # loop through the whole rotated test split
            for images, labels in rotated_test_loader:
                # predictions from my inference helper
                predictions = inference(model, images, labels)

                # count how many are right
                correct += (predictions == labels.to(device)).sum().item()
                total += labels.size(0)

            # accuracy for this one specific angle
            accuracy = correct / total
            angle_accuracies.append(accuracy)

        
            #######################
            # END OF YOUR CODE    #
            #######################
    
    return angles, angle_accuracies          

In [ ]:
angles, angle_accuracies = get_acc_per_angle(model)

In [ ]:
# Plot accuracy vs rotation angle
def plot(angles, angle_accuracies):
            #######################
            # PUT YOUR CODE HERE  #
            #######################    

        plt.figure(figsize=(7, 5))  # making the figure a bit wider so it's readable

        # plotting the angle on x-axis and the measured accuracy on y-axis
        plt.plot(angles, angle_accuracies, marker='o')

        plt.xlabel("Rotation angle (degrees)")
        plt.ylabel("Accuracy")

        # simple title, nothing fancy
        plt.title("Accuracy vs rotation angle")

        # I like to keep the grid on, otherwise it’s hard to see trends
        plt.grid(True)

        # make sure every tested angle shows up on the x ticks
        plt.xticks(angles)

        # accuracy should always be between 0 and 1, so I force the limits
        plt.ylim(0, 1.0)

        # finally show the plot
        plt.show()



            #######################
            # END OF YOUR CODE    #
            #######################

In [ ]:
angles, angle_accuracies = get_acc_per_angle(model)
plot(angles, angle_accuracies)

### Question 1.2 (b)

As said in the pdf, now we will first train a model with the same architecture as the previous one (this is, you can use the same to initialize the model as before), changing the train dataset so that it contains _random rotations_ of angles of up to 360 degrees. For this, create a new `train_augmentation_transform` to create the train augmented dataset.

Hint: Check how we used transformations in Question 1.2 (a)

In [ ]:
# Create the new train data and loader and visualize it

#######################
# PUT YOUR CODE HERE  #
#######################
# for the training data I'm adding some random rotations, basically letting the model
# see the images in many different orientations,hoping it helps it generalize better
train_augmentation_transform = transforms.Compose([
    transforms.RandomRotation(degrees=(0, 360)),  # full rotation range
    transforms.ToTensor(),  # convert the image into a PyTorch tensor
])

#######################
# END OF YOUR CODE    #
#######################

train_augmented_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_augmentation_transform)
train_augmented_loader = torch.utils.data.DataLoader(train_augmented_data, batch_size=64, shuffle=True)


display_first_few_images(train_augmented_data)

As said, we will now initialize the new model and train it over the `train_augmented_loader` you just created. Initialized the new model as we did in the previous question and train it using the `train` function. 

Note: Again, you shouldn't need Snellius to run it 

In [ ]:
#######################
# PUT YOUR CODE HERE  #
#######################

model_aug = CNN().to(device)

#training model

train(model_aug, train_augmented_loader)
#######################
# END OF YOUR CODE    #
#######################

Now, evaluate its performance by running inference over the dataset when rotating the images 0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330 and 360 degrees, and plotting the model's accuracy respect to the angle of rotation of the test dataset. You can use `get_acc_per_angle` and `plot` functions you defined in Question 1.2 (a)!

Hint: The test data is the same as in Question 1.2 (a)

In [ ]:
#######################
# PUT YOUR CODE HERE  #
#######################

angles_aug, angle_accuracies_aug = get_acc_per_angle(model_aug)

#calling plots4

plot(angles_aug, angle_accuracies_aug)
#######################
# END OF YOUR CODE    #
#######################